In [1]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
from PIL import Image
from tqdm import tqdm
from torch.utils.data import DataLoader
from utils.utils import *

random.seed(1)

# Autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
data_folder = "data"

image_files = [f for f in os.listdir(data_folder) if f.endswith("png")]

### Chose a random image to train a neural representation on

In [ ]:
# Choose a random image to train on
img_file_path = '0002.png'
img_path = os.path.join(data_folder, img_file_path)
img_original = Image.open(img_path)

# Convert image to numpy array
img_np_original = np.array(img_original)

# Display the original image
plt.figure(figsize=(10, 10))
plt.imshow(img_np_original)
plt.axis("off")
plt.title(f"Original Image: {img_file_path}")
plt.show()

# Get image dimensions
height_target, width_target, channels = img_np_original.shape
print(f"Image dimensions: {height_target}x{width_target}, {channels} channels")

In [ ]:
# Now we will downsize the original image to a smaller resolution using a random integer factor
downsize_factor = 3
print(f"Downsize factor: {downsize_factor}")
img_resized = img_original.resize((width_target // downsize_factor, height_target // downsize_factor))

# convert image to numpy array
img_np_resized = np.array(img_resized)

# Display the resized image
plt.figure(figsize=(10, 10))
plt.imshow(img_np_resized)
plt.axis("off")
plt.title(f"Resized Image: {width_target // downsize_factor}x{height_target // downsize_factor}")
plt.show()

# Print the resolution of the new image
height_resized, width_resized, channels = img_np_resized.shape
print(f"Resized image dimensions: {height_resized}x{width_resized}, {channels} channels")

### We will next normalize the pixels to have values between 0 and 1

In [6]:
# Normalize the resized image
img_np_resized_standardized = img_np_resized / 255.0  # Normalize to [0, 1]

In [ ]:
# Print the mean of the resized image
print(f"Resized image mean: {np.mean(img_np_resized_standardized, axis=(0, 1))}")
print(f"Resized image std: {np.std(img_np_resized_standardized, axis=(0, 1))}")

### Normalize the coordinates of the image to be between 0 and 1.

In [ ]:
normalized_coordinates = get_normalized_coordinates(height_resized, width_resized)
pixel_values = img_np_resized_standardized.reshape(-1, channels)

# Check shapes
print(f"Normalized coordinates shape: {normalized_coordinates.shape}")
print(f"Pixel values shape: {pixel_values.shape}")

### Create the dataset and dataloader for the downsampled image

In [9]:
# Create dataset and dataloader
dataset = ImageDataset(normalized_coordinates, pixel_values)
dataloader = DataLoader(dataset, batch_size=4096, shuffle=True)

In [ ]:
# Create and train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.MSELoss()
model = INRModel(input_dim=2, output_dim=channels, hidden_dim=128, num_layers=3, dropout_rate=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
losses = model.train_model(dataloader, num_epochs=100, lr=1e-3, device=device, criterion = criterion, optimizer = optimizer)

# Plot the loss curve
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(True)
plt.show()

In [ ]:
# Generate normalized coordinates for the original image size
target_coords_normalized = get_normalized_coordinates(height_target, width_target)

# Convert to PyTorch tensor
target_coords_tensor = torch.from_numpy(target_coords_normalized).float().to(device)

# Use the trained model to predict the RGB values for these coordinates
with torch.no_grad():
    pred_pixels = model(target_coords_tensor).cpu().numpy()
    
print(f"Max and Min values of pred_pixels: {np.max(pred_pixels)}, {np.min(pred_pixels)}")

# Denormalize the RGB values using the mean and std of the resized image (i.e the training data)
reconstructed_img_reshaped_normalized = pred_pixels.reshape(height_target, width_target, channels)
# Convert to integer type and denormalize
reconstructed_img_reshaped_denormalized = (reconstructed_img_reshaped_normalized * 255).astype(np.uint8)
# IMPORTANT: Need to round the pixels to the nearest integer value and clip them to [0, 255] range
reconstructed_img_reshaped_denormalized = np.clip(reconstructed_img_reshaped_denormalized, 0, 255)

# Display the reconstructed image
plt.figure(figsize=(10, 10))
plt.imshow(reconstructed_img_reshaped_denormalized)
plt.title('Reconstructed Image at Original Size')
plt.axis('off')
plt.show()

# Check that the shapes of the reconstructed image and original image are the same

print(f"Downsized image shape: {img_np_resized.shape}")
print(f"Reconstructed image shape: {reconstructed_img_reshaped_denormalized.shape}")
print(f"Original image shape: {img_np_original.shape}")